In [1]:
! ls -ltr

total 5660
-rw-r--r-- 1 atif atif 5769361 Nov 14 21:17 weatherHistory.csv
-rw-r--r-- 1 atif atif   14449 Nov 14 22:39 Handling_missing_values.ipynb
drwxr-xr-x 2 atif atif    4096 Nov 14 22:40 valid
drwxr-xr-x 2 atif atif    4096 Nov 14 22:40 invalid


In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local").appName("Missing_value Handling")\
.getOrCreate()

In [3]:
out_valid_path ="valid"
out_invalid_path ="invalid"


In [4]:
df_in = spark.read.csv("weatherHistory.csv",header=True)

In [5]:
#caching df_in
df_in.cache()

DataFrame[DateTime: string, Temperature (C): string, Humidity: string, Wind Speed (km/h): string, Pressure (millibars): string, Summary: string]

In [6]:
df_in.show()

+---------------+---------------+--------+-----------------+--------------------+-------------+
|       DateTime|Temperature (C)|Humidity|Wind Speed (km/h)|Pressure (millibars)|      Summary|
+---------------+---------------+--------+-----------------+--------------------+-------------+
|4/1/06 12:00 AM|    9.472222222|    0.89|          14.1197|             1015.13|Partly Cloudy|
| 4/1/06 1:00 AM|    9.355555556|    0.86|          14.2646|             1015.63|Partly Cloudy|
| 4/1/06 2:00 AM|    9.377777778|    0.89|           3.9284|             1015.94|Mostly Cloudy|
| 4/1/06 3:00 AM|    8.288888889|    0.83|          14.1036|                null|Partly Cloudy|
| 4/1/06 4:00 AM|    8.755555556|    0.83|          11.0446|             1016.51|Mostly Cloudy|
| 4/1/06 5:00 AM|    9.222222222|    0.85|          13.9587|             1016.66|Partly Cloudy|
| 4/1/06 6:00 AM|    7.733333333|    0.95|          12.3648|             1016.72|Partly Cloudy|
| 4/1/06 7:00 AM|    8.772222222|    0.8

In [7]:
df_valid =df_in.filter("`Temperature (C)` is not null and Humidity is not null and \
                       `Wind Speed (km/h)` is not null and `Pressure (millibars)` is not null \
                       and Summary is not null")

In [8]:
df_valid.show()

+---------------+---------------+--------+-----------------+--------------------+-------------+
|       DateTime|Temperature (C)|Humidity|Wind Speed (km/h)|Pressure (millibars)|      Summary|
+---------------+---------------+--------+-----------------+--------------------+-------------+
|4/1/06 12:00 AM|    9.472222222|    0.89|          14.1197|             1015.13|Partly Cloudy|
| 4/1/06 1:00 AM|    9.355555556|    0.86|          14.2646|             1015.63|Partly Cloudy|
| 4/1/06 2:00 AM|    9.377777778|    0.89|           3.9284|             1015.94|Mostly Cloudy|
| 4/1/06 4:00 AM|    8.755555556|    0.83|          11.0446|             1016.51|Mostly Cloudy|
| 4/1/06 5:00 AM|    9.222222222|    0.85|          13.9587|             1016.66|Partly Cloudy|
| 4/1/06 6:00 AM|    7.733333333|    0.95|          12.3648|             1016.72|Partly Cloudy|
| 4/1/06 7:00 AM|    8.772222222|    0.89|          14.1519|             1016.84|Partly Cloudy|
| 4/1/06 8:00 AM|    10.82222222|    0.8

In [9]:
df_invalid =df_in.filter("`Temperature (C)` is  null or Humidity is null or  \
                       `Wind Speed (km/h)` is  null or `Pressure (millibars)` is  null \
                       or Summary is  null")

In [10]:
df_invalid.show()

+-----------------+---------------+--------+-----------------+--------------------+-------------+
|         DateTime|Temperature (C)|Humidity|Wind Speed (km/h)|Pressure (millibars)|      Summary|
+-----------------+---------------+--------+-----------------+--------------------+-------------+
|   4/1/06 3:00 AM|    8.288888889|    0.83|          14.1036|                null|Partly Cloudy|
|  4/1/06 12:00 PM|           null|    0.55|          21.9443|             1017.59|Partly Cloudy|
|   4/1/06 5:00 PM|          15.55|    0.63|             null|             1016.17|Mostly Cloudy|
|  4/1/06 11:00 PM|           10.2|    null|           3.9284|             1015.51|Mostly Cloudy|
| 4/20/06 11:00 PM|           null|    null|             null|             1016.38|Mostly Cloudy|
| 7/28/07 11:00 PM|           null|    null|             null|                null|Mostly Cloudy|
|  7/29/07 9:00 AM|           null|    0.55|          20.6885|             1014.65|Mostly Cloudy|
|12/17/08 12:00 PM| 

In [11]:
# Writing the valid and invalid records to respetive directory

In [12]:
df_valid.write.mode('overwrite').format('csv').option("header",True).save(out_valid_path)

In [13]:
df_invalid.write.mode('overwrite').format('csv').option("header",True).save(out_invalid_path)

### Partition

In [14]:
from pyspark.sql.functions import to_date , from_unixtime,unix_timestamp
from pyspark.sql.types import TimestampType

In [15]:
df_valid.select('DateTime').show()

+---------------+
|       DateTime|
+---------------+
|4/1/06 12:00 AM|
| 4/1/06 1:00 AM|
| 4/1/06 2:00 AM|
| 4/1/06 4:00 AM|
| 4/1/06 5:00 AM|
| 4/1/06 6:00 AM|
| 4/1/06 7:00 AM|
| 4/1/06 8:00 AM|
| 4/1/06 9:00 AM|
|4/1/06 10:00 AM|
|4/1/06 11:00 AM|
| 4/1/06 1:00 PM|
| 4/1/06 2:00 PM|
| 4/1/06 3:00 PM|
| 4/1/06 4:00 PM|
| 4/1/06 6:00 PM|
| 4/1/06 7:00 PM|
| 4/1/06 8:00 PM|
| 4/1/06 9:00 PM|
|4/1/06 10:00 PM|
+---------------+
only showing top 20 rows



In [19]:
df_valid = df_valid.withColumn("DateOnly",to_date(
from_unixtime(
unix_timestamp("DateTime","MM/dd/yy hh:mm a")
).cast(TimestampType())))

In [20]:
df_valid.show(3)

Py4JJavaError: An error occurred while calling o85.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 9.0 failed 1 times, most recent failure: Lost task 0.0 in stage 9.0 (TID 9, 192.168.0.111, executor driver): org.apache.spark.SparkUpgradeException: You may get a different result due to the upgrading of Spark 3.0: Fail to parse '4/1/06 12:00 AM' in the new parser. You can set spark.sql.legacy.timeParserPolicy to LEGACY to restore the behavior before Spark 3.0, or set to CORRECTED and treat it as an invalid datetime string.
	at org.apache.spark.sql.catalyst.util.DateTimeFormatterHelper$$anonfun$checkParsedDiff$1.applyOrElse(DateTimeFormatterHelper.scala:150)
	at org.apache.spark.sql.catalyst.util.DateTimeFormatterHelper$$anonfun$checkParsedDiff$1.applyOrElse(DateTimeFormatterHelper.scala:141)
	at scala.runtime.AbstractPartialFunction.apply(AbstractPartialFunction.scala:38)
	at org.apache.spark.sql.catalyst.util.Iso8601TimestampFormatter.$an